In [1]:
#Import der benötigten Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import librosa
import librosa.display
from IPython.display import Audio
from sklearn.model_selection import train_test_split
import sklearn.preprocessing as skp
import tensorflow as tf
from tensorflow import keras as k
import math

In [2]:
os_dir = os.getcwd()
path_to_soundfilefolder = f'{os_dir}\\Datensatz\\processed_audio'
print(path_to_soundfilefolder)

E:\Notebooks\Genre_Classification\Datensatz\processed_audio


In [ ]:
#Zeug von Librosa
#y_harmonic, y_percussive = librosa.effects.hpss(y)  -------> noch beschreiben, was die einzelnen Attribute aussagen
#tempo, beat_frames = librosa.beat.beat_track(y=y_percussive, sr=sr)
#mfcc = librosa.feature.mfcc(y=y, sr=sr, hop_length=hop_length, n_mfcc=13)
#mfcc_delta = librosa.feature.delta(mfcc)
#beat_mfcc_delta = librosa.util.sync(np.vstack([mfcc, mfcc_delta]), beat_frames)
#chromagram = librosa.feature.chroma_cqt(y=y_harmonic, sr=sr)
#beat_chroma = librosa.util.sync(chromagram, beat_frames, aggregate=np.median)
#beat_features = np.vstack([beat_chroma, beat_mfcc_delta])

In [86]:
#Import der Sound-Dateien\n
import os
import numpy as np

#Zuordnung der Labels zu Zahlen
genre_names = []

#enthält sämtliche Einträge
features = []
#enthält die Ergebnisse sämtlicher Einträge
genre = []

print(path_to_soundfilefolder)

for filename in os.listdir(path_to_soundfilefolder):
    #hier kommen alle Attribute pro Soundfile rein
    attributes = []
    
    #Genre einfügen
    attributes.append(filename.split("_")[0])
    
    #Umwandlung der Soundfiles in repräsentative Zahlenwerte
    print(f"Converting: {filename}")
    y, sr = librosa.load(f'{os_dir}\\Datensatz\\processed_audio\\' + filename)
    
    #extrahieren der aussagekräftigen Eigenschaften
    stft_array = librosa.feature.chroma_stft(y=y, sr=sr)
    stft_mean = np.mean(stft_array)
    stft_var = np.var(stft_array)

    cq_array = librosa.feature.chroma_cqt(y=y, sr=sr)
    cq_mean = np.mean(cq_array)
    cq_var = np.var(cq_array)

    rms_array = librosa.feature.rms(y=y)
    rms_mean = np.mean(rms_array)
    rms_var = np.var(rms_array)

    cent_array = librosa.feature.spectral_centroid(y=y, sr=sr)
    cent_mean = np.mean(cent_array)
    cent_var = np.var(cent_array)

    spec_array = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    spec_mean = np.mean(spec_array)
    spec_var = np.var(spec_array)

    rolloff_array = librosa.feature.spectral_rolloff(y=y, sr=sr)
    rolloff_mean = np.mean(rolloff_array)
    rolloff_var = np.var(rolloff_array)

    zero_crossing_rate_array = librosa.feature.zero_crossing_rate(y)
    zero_crossing_rate_mean = np.mean(zero_crossing_rate_array)
    zero_crossing_rate_var = np.var(zero_crossing_rate_array)

    tonnetz_array = librosa.feature.tonnetz(y=y, sr=sr)
    tonnetz_mean = np.mean(tonnetz_array)
    tonnetz_var = np.var(tonnetz_array)

    y_harmonic_array, y_percussive = librosa.effects.hpss(y)
    tempo1, beat_frames = librosa.beat.beat_track(y=y_percussive, sr=sr)
    tempo = np.float32(tempo1)
    
    y_harmonic_mean = np.mean(y_harmonic_array)
    y_harmonic_var = np.var(y_harmonic_array)

    attributes.extend([stft_mean, stft_var, cq_mean, cq_var, rms_mean, rms_var, cent_mean, cent_var, spec_mean, spec_var, rolloff_mean, rolloff_var, zero_crossing_rate_mean, zero_crossing_rate_var, tonnetz_mean, tonnetz_var, tempo, y_harmonic_mean, y_harmonic_var]) 

    #das mfcc-array besteht aus 20 arrays, für die jeweils mean und var bestimmt werden
    mfcc_array = librosa.feature.mfcc(y=y, sr=sr)
    for i in range(20):
        mfcc_mean = np.mean(mfcc_array[i])
        mfcc_var = np.var(mfcc_array[i])
        attributes.extend([mfcc_mean, mfcc_var])  
        
    
    
    #Anhängen eines Eintrages
    features.append(attributes)
    
    

E:\Notebooks\Genre_Classification\Datensatz\processed_audio
Converting: blues_100_0.wav
Converting: blues_100_1.wav
Converting: blues_100_2.wav
Converting: blues_100_3.wav
Converting: blues_100_4.wav
Converting: blues_100_5.wav
Converting: blues_100_6.wav
Converting: blues_100_7.wav
Converting: blues_100_8.wav
Converting: blues_100_9.wav
Converting: blues_10_0.wav
Converting: blues_10_1.wav
Converting: blues_10_2.wav
Converting: blues_10_3.wav
Converting: blues_10_4.wav
Converting: blues_10_5.wav
Converting: blues_10_6.wav
Converting: blues_10_7.wav
Converting: blues_10_8.wav
Converting: blues_10_9.wav
Converting: blues_11_0.wav
Converting: blues_11_1.wav
Converting: blues_11_2.wav
Converting: blues_11_3.wav
Converting: blues_11_4.wav
Converting: blues_11_5.wav
Converting: blues_11_6.wav
Converting: blues_11_7.wav
Converting: blues_11_8.wav
Converting: blues_11_9.wav


KeyboardInterrupt: 

In [87]:
#Array an konvertierten Eigenschaften in CSV speichern
arr = features # Hier muss dann das Ergebnis Array rein
headers = ["genre","stft_mean", "stft_var", "cq_mean", "cq_var", "rms_mean", "rms_var", "cent_mean", "cent_var", "spec_mean", "spec_var", "rolloff_mean", "rolloff_var", "zero_crossing_rate_mean", "zero_crossing_rate_var", "tonnetz_mean", "tonnetz_var", "tempo", "y_harmonic_mean", "y_harmonic_var", "mfcc_mean_1", "mfcc_var_1", "mfcc_mean_2", "mfcc_var_2", "mfcc_mean_3", "mfcc_var_3", "mfcc_mean_4", "mfcc_var_4", "mfcc_mean_5", "mfcc_var_5", "mfcc_mean_6", "mfcc_var_6", "mfcc_mean_7", "mfcc_var_7", "mfcc_mean_8", "mfcc_var_8", "mfcc_mean_9", "mfcc_var_9", "mfcc_mean_10", "mfcc_var_10", "mfcc_mean_11", "mfcc_var_11", "mfcc_mean_12", "mfcc_var_12", "mfcc_mean_13", "mfcc_var_13", "mfcc_mean_14", "mfcc_var_14", "mfcc_mean_15", "mfcc_var_15", "mfcc_mean_16", "mfcc_var_16", "mfcc_mean_17", "mfcc_var_17", "mfcc_mean_18", "mfcc_var_18", "mfcc_mean_19", "mfcc_var_19", "mfcc_mean_20", "mfcc_var_20"] # Das sind die Überschriften
df = pd.DataFrame(arr) 
pd.DataFrame(df).to_csv(f"{os_dir}\Datensatz\\results.csv", header = headers, index=False)

In [111]:
df = pd.read_csv(f'Datensatz//results.csv')
song_labels = df['genre']

label_index = dict()

for i, x in enumerate(df.genre.unique()):
    label_index[x] = i

labels = []

for x in range(len(song_labels)):
    labels.append(label_index[song_labels[x]])
    
genre_list = pd.DataFrame(labels)

features = df.iloc[0:, 1:]


In [115]:
# normalize
cols = features.columns
min_max_scaler = skp.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(features)
features_normalized = pd.DataFrame(np_scaled, columns = cols)

In [116]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(features_normalized,genre_list,test_size = 0.2,random_state = 42)

scaler = skp.StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
X_test = pd.DataFrame(scaler.transform(X_test), columns=X_train.columns)


In [117]:
#Programmierung des Modells
import tensorflow as tf
from tensorflow import keras

model = keras.Sequential([
    keras.layers.Dense(64, activation=tf.nn.relu),
    keras.layers.Dense(32, activation=tf.nn.relu),
    keras.layers.Dense(16, activation=tf.nn.relu),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])

model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.fit(X_train, Y_train, epochs=300)
model.summary()

Epoch 1/300
1/1 [==============================] - 0s 347ms/step - loss: 2.1959 - accuracy: 0.1304
Epoch 2/300
1/1 [==============================] - 0s 3ms/step - loss: 2.0951 - accuracy: 0.1304
Epoch 3/300
1/1 [==============================] - 0s 3ms/step - loss: 1.9956 - accuracy: 0.1739
Epoch 4/300
1/1 [==============================] - 0s 3ms/step - loss: 1.8974 - accuracy: 0.2609
Epoch 5/300
1/1 [==============================] - 0s 4ms/step - loss: 1.8015 - accuracy: 0.4348
Epoch 6/300
1/1 [==============================] - 0s 4ms/step - loss: 1.7050 - accuracy: 0.6087
Epoch 7/300
1/1 [==============================] - 0s 3ms/step - loss: 1.6090 - accuracy: 0.6957
Epoch 8/300
1/1 [==============================] - 0s 3ms/step - loss: 1.5159 - accuracy: 0.8261
Epoch 9/300
1/1 [==============================] - 0s 4ms/step - loss: 1.4240 - accuracy: 0.8261
Epoch 10/300
1/1 [==============================] - 0s 4ms/step - loss: 1.3391 - accuracy: 0.8696
Epoch 11/300
1/1 [=========

1/1 [==============================] - 0s 5ms/step - loss: 0.0032 - accuracy: 1.0000
Epoch 85/300
1/1 [==============================] - 0s 4ms/step - loss: 0.0032 - accuracy: 1.0000
Epoch 86/300
1/1 [==============================] - 0s 5ms/step - loss: 0.0031 - accuracy: 1.0000
Epoch 87/300
1/1 [==============================] - 0s 3ms/step - loss: 0.0030 - accuracy: 1.0000
Epoch 88/300
1/1 [==============================] - 0s 4ms/step - loss: 0.0030 - accuracy: 1.0000
Epoch 89/300
1/1 [==============================] - 0s 3ms/step - loss: 0.0029 - accuracy: 1.0000
Epoch 90/300
1/1 [==============================] - 0s 3ms/step - loss: 0.0028 - accuracy: 1.0000
Epoch 91/300
1/1 [==============================] - 0s 7ms/step - loss: 0.0028 - accuracy: 1.0000
Epoch 92/300
1/1 [==============================] - 0s 2ms/step - loss: 0.0027 - accuracy: 1.0000
Epoch 93/300
1/1 [==============================] - 0s 2ms/step - loss: 0.0027 - accuracy: 1.0000
Epoch 94/300
1/1 [===============

1/1 [==============================] - 0s 2ms/step - loss: 0.0010 - accuracy: 1.0000
Epoch 167/300
1/1 [==============================] - 0s 3ms/step - loss: 9.9925e-04 - accuracy: 1.0000
Epoch 168/300
1/1 [==============================] - 0s 2ms/step - loss: 9.8897e-04 - accuracy: 1.0000
Epoch 169/300
1/1 [==============================] - 0s 2ms/step - loss: 9.7887e-04 - accuracy: 1.0000
Epoch 170/300
1/1 [==============================] - 0s 2ms/step - loss: 9.6892e-04 - accuracy: 1.0000
Epoch 171/300
1/1 [==============================] - 0s 2ms/step - loss: 9.5908e-04 - accuracy: 1.0000
Epoch 172/300
1/1 [==============================] - 0s 2ms/step - loss: 9.4942e-04 - accuracy: 1.0000
Epoch 173/300
1/1 [==============================] - 0s 2ms/step - loss: 9.3986e-04 - accuracy: 1.0000
Epoch 174/300
1/1 [==============================] - 0s 5ms/step - loss: 9.3047e-04 - accuracy: 1.0000
Epoch 175/300
1/1 [==============================] - 0s 2ms/step - loss: 9.2126e-04 - accur

1/1 [==============================] - 0s 2ms/step - loss: 5.0704e-04 - accuracy: 1.0000
Epoch 246/300
1/1 [==============================] - 0s 2ms/step - loss: 5.0337e-04 - accuracy: 1.0000
Epoch 247/300
1/1 [==============================] - 0s 2ms/step - loss: 4.9968e-04 - accuracy: 1.0000
Epoch 248/300
1/1 [==============================] - 0s 6ms/step - loss: 4.9607e-04 - accuracy: 1.0000
Epoch 249/300
1/1 [==============================] - 0s 2ms/step - loss: 4.9248e-04 - accuracy: 1.0000
Epoch 250/300
1/1 [==============================] - 0s 3ms/step - loss: 4.8893e-04 - accuracy: 1.0000
Epoch 251/300
1/1 [==============================] - 0s 3ms/step - loss: 4.8541e-04 - accuracy: 1.0000
Epoch 252/300
1/1 [==============================] - 0s 3ms/step - loss: 4.8197e-04 - accuracy: 1.0000
Epoch 253/300
1/1 [==============================] - 0s 2ms/step - loss: 4.7855e-04 - accuracy: 1.0000
Epoch 254/300
1/1 [==============================] - 0s 2ms/step - loss: 4.7512e-04 - a

In [ ]:
#bestes Modell aus dem Internet
model1 = k.models.Sequential([
    k.layers.Dense(1024, activation='relu', input_shape=(x_train.shape[1],)),
    k.layers.Dropout(0.3),
    
    k.layers.Dense(512, activation='relu'),
    k.layers.Dropout(0.3),

    k.layers.Dense(256, activation='relu'),
    k.layers.Dropout(0.3),

    k.layers.Dense(128, activation='relu'),
    k.layers.Dropout(0.3),

    k.layers.Dense(64, activation='relu'),
    k.layers.Dropout(0.3),

    k.layers.Dense(10, activation='softmax'),
])
print(model1.summary())

model1.compile(optimizer='rmsprop',
                  loss='sparse_categorical_crossentropy',
                  metrics='accuracy')

model1.fit(x_train, y_train, epochs=300)

In [ ]:
import numpy as np
import sklearn.metrics

predictions_confidences = model.predict(x_test)

predictions = []
for entry in predictions_confidences:
        predictions.append(np.argmax(entry))
print(predictions)

In [ ]:
print(f"{sklearn.metrics.classification_report(y_test, predictions)}\n")

In [ ]:
#test librosa
y, sr = librosa.load(f'{os_dir}\\Datensatz\\processed_audio\\blues_1_0.wav')
print(y, sr)

In [ ]:
tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr)
print(tempo, beat_frames)

In [ ]:
# Backup Für Standard CSV
df = pd.read_csv(f'Datensatz//features_3_sec.csv')

data = df.iloc[0:, 1:]

song_labels = data['label']
label_index = dict()

for i, x in enumerate(df.label.unique()):
    label_index[x] = i

labels = []

#print(len(song_labels))
for x in range(len(song_labels)):
    labels.append(label_index[song_labels[x]])

genre_list = pd.DataFrame(labels)

# normalize
cols = features.columns
min_max_scaler = skp.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(features)
features_normalized = pd.DataFrame(np_scaled, columns = cols)
